In [1]:
from langchain_community.llms import Ollama
import json
import random
import re

# Combine several papers

In [26]:
def get_json(file_name):
    # Step 1: Read the JSON file
    with open(file_name + '.json', 'r') as file:
        json_data = json.load(file)
    return json_data
def write_json(dict_data,file_name):
    with open('summaries/'+file_name+'.json', 'w') as file:
        json.dump(dict_data, file, indent=4)


def store_summary(survey1,filename):
    with open(f'summaries/'+filename+'.txt', 'w') as file:
        file.write(survey1)

In [3]:

# The original state of the art paper
paper_id = "2402.01383v1"
original = get_json('dataset/'+paper_id+'data')
topic = original['title']
topic

'LLM-based NLG Evaluation: Current Status and Challenges'

In [9]:
#Get full texts of related papers
data = get_json('dataset/'+paper_id+'full_texts')

In [10]:
# Get the llm mistral
llm = Ollama(model = "mistral",temperature=0)

In [11]:
def summarize_n(papers):
    l = [key+':  '+value for key,value in papers.items()]
    random.shuffle(l)
    context = ('/n/n').join(l)
    instruction = 'summarize the following papers:/n'+context+'/n summarize them.'
    a = llm.invoke(instruction)
    return a

In [12]:
longsummary = summarize_n(data)
longsummary
#type(random.shuffle([key+':  '+value for key,value in data.items()]))
#complete fail

' In response to the user question "What are the most effective ways to deal with stress?", Assistant 1 provided an answer that contained information about effective ways to manage stress, such as regular exercise, maintaining a healthy diet, getting enough sleep, and engaging in relaxation activities like meditation, as well as having a support system. The response was clear, concise, and relevant to the question.\n\nAssistant 2\'s answer also provided valid suggestions for dealing with stress, including taking deep breaths, regular physical exercise, maintaining a balanced diet, and finding hobbies or activities that bring enjoyment. However, there were several grammatical errors in the response, such as "Whenevr feeling stressed" instead of "When feeling stressed", "always remember to take deep breathes" instead of "remember to take deep breaths when feeling stressed", and "This can calm nerves" instead of "Taking deep breaths can calm nerves". Additionally, there were some minor fa

In [16]:
#Is the problem that context is too big? 
l = [key+':  '+value for key,value in data.items()]
context = ('/n/n').join(l)
len(context) #3'924'350 characters. This is way too big as 
len(context.split(' ')) # 499'106 words but only 128'000 in theory are allowed. The llm was trained with 8'000 words in the prompt.
#yes

499106

Try with only abstracts. Is it short enough?

In [17]:
#Get full texts of related papers
data = get_json('dataset/'+paper_id+'ref')

In [22]:
l = [value['title']+':  '+value['abstract'] for value in data.values()]
context = ('/n/n').join(l)
len(context.split(' ')) 
#7956 is short enough

7956

In [24]:
def write_summary(summaries):
    texts = ('\n').join([s['title']+':  '+s['abstract'] for s in summaries.values()])
    instruction = f'Write a summary combining the key findings of following texts: \n {texts}'
    a = llm.invoke(instruction)
    return a

In [25]:
simple_summary = write_summary(data)
simple_summary

' This text discusses several research papers related to evaluating and using large language models (LLMs) as judges for various tasks. The first paper explores the use of wider and deeper networks for fairer LLM evaluations, drawing inspiration from deep neural networks and academic paper reviewing. The second paper proposes using strong LLMs as judges for evaluating other LLMs on more open-ended questions and introduces two benchmarks: MT-bench and Chatbot Arena. The third paper proposes fine-tuning LLMs as scalable judges (JudgeLM) to evaluate LLMs efficiently and effectively in open-ended benchmarks.\n\nThe first paper, "Wider and Deeper LLM Networks are Fairer LLM Evaluators," discusses the challenges of evaluating the quality of responses generated by LLMs and proposes a method using an LLM itself to make evaluations through multiple independent evaluations. The authors explore whether deeper and wider networks can lead to fairer evaluations, inspired by the observation that diff

In [27]:
store_summary(simple_summary,'simple_summary')

In [32]:
print(('\n').join([s['title'] for s in data.values()]))
#The first paper, "Wider and Deeper LLM Networks are Fairer LLM Evaluators,"... 
#=> it did only consider the last three papers

Concrete Problems in AI Safety
PaLM 2 Technical Report
Benchmarking Foundation Models with Language-Model-as-an-Examiner
ChatEval: Towards Better LLM-based Evaluators through Multi-Agent Debate
BooookScore: A systematic exploration of book-length summarization in the era of LLMs
Scaling Instruction-Finetuned Language Models
RAGAS: Automated Evaluation of Retrieval Augmented Generation
GPTScore: Evaluate as You Desire
Are Large Language Models Reliable Judges? A Study on the Factuality Evaluation Capabilities of LLMs
CoAScore: Chain-of-Aspects Prompting for NLG Evaluation
ALLURE: Auditing and Improving LLM-based Evaluation of Text using Iterative In-Context-Learning
SocREval: Large Language Models with the Socratic Method for Reference-Free Reasoning Evaluation
Exploring ChatGPT's Ability to Rank Content: A Preliminary Study on Consistency with Human Preferences
TIGERScore: Towards Building Explainable Metric for All Text Generation Tasks
CritiqueLLM: Towards an Informative Critique Gen

In [33]:
def write_summary_with_topic(summaries,topic):
    texts = ('\n').join([s['title']+':  '+s['abstract'] for s in summaries.values()])
    instruction = f'Use this {texts} to create an overview over the topic {topic}.'
    a = llm.invoke(instruction)
    return a

In [34]:
simple_overview = write_summary_with_topic(data,topic)
store_summary(simple_overview,'simple_overview')
simple_overview

' Title: LLM-based NLG Evaluation: Current Status and Challenges - Wider and Deeper Networks for Fairer Evaluators, LLM-as-a-Judge, and JudgeLM\n\nThis topic covers recent research on evaluating Large Language Models (LLMs) in Natural Language Generation (NLG) tasks. The studies explore various approaches to improve the evaluation process, including wider and deeper networks for fairer evaluators, using LLMs as judges, and fine-tuning LLMs as scalable judges.\n\n1. Wider and Deeper LLM Networks are Fairer LLM Evaluators:\nThis paper proposes a novel approach to use the LLM itself to make evaluations and improve fairness by designing a network that resembles academic paper reviewing. The network consists of multiple layers, with each layer receiving representations from all neurons in the previous layer, integrating locally learned evaluation information for more comprehensive results. Experimental results demonstrate that a wider network (involving many reviewers) with 2 layers perform